In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo", ]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

In [ ]:
from PIL import Image
from pathlib import Path, PurePath
import re
import numpy as np
from math import cos, sin, radians
import decimal
import matplotlib.pyplot as plt

In [ ]:
def read_lo_res_img_set(img_set_path, n_leds, **kwargs):
    """Reads the files of the Lo-Res image set and returns all of them
    as a 3-dimentional array (numpy.ndarray).
    """
    img_prefix = kwargs.pop('prefix', 'IMG_')
    img_format = kwargs.pop('format', 'PNG')
    p = Path(img_set_path)
    str_n_leds = str(n_leds)
    re_wide_num_range = '[0-' + str_n_leds[0] + ']'
    for i in range(len(str_n_leds) - 1):
         re_wide_num_range += '[0-9]'
    # Creates a list with the files within p (Path) whose name begins
    # with glob_pattern (string). glob() does not support regexp.
    glob_pattern = img_prefix + re_wide_num_range + '.*'
    file_list = list(p.glob(glob_pattern))
    file_list.sort()  # Sort the file list.
    # Create a regexp and use it to filter file_list (list).
    str_regex = img_prefix + '(?!0+\.)'
    if len(str_n_leds) > 1:
        str_regex += '(?!' + str_n_leds[0]
        for i in range(1, len(str_n_leds)):
            str_regex += '[' + str(int(str_n_leds[i]) + 1) + '-9]'
        str_regex += ')'
    str_regex += re_wide_num_range
    str_regex += '.('+ img_format.lower() + '|'
    str_regex += img_format.upper() + ')'
    filtered_file_list = list()
    img_cube = list()
    # Convert from regexp pattern to regexp object.
    regexp = re.compile(str_regex)
    # Store the images as 2-dimensionals arrays within a 3-dimensional
    # array.
    for item in file_list:
        m = regexp.search(str(item))
        if m and str(item).endswith(m.group(0)):
            img_path = str(item)
            img_num = m.group(0).split('_')[1].split('.')[0]
            print('<FP-ImageRead-' + img_num + '>')
            img = Image.open(img_path)
            img.close()
            arr_img = np.array(img)
            img_cube.append(arr_img)
    return np.array(img_cube)

In [ ]:
def get_pixels_xy(**kwargs):
    """Calculates the (x, y) coordinates of the Pixels in a ring or
    Jewel and stores them a list of tuple within a dictionary.
    """
    rings = kwargs.pop('rings', {7: 0, 12: 0, 24: 0});
    cm = kwargs.pop('cm', True)  # (x, y) in centimeters?
    # The properties for each NeoPixel ring or Jewel were retrieved from
    # https://learn.adafruit.com/adafruit-neopixel-uberguide/downloads
    NEOPIXEL_LED_RINGS = {
        '7': {
            'name': 'NeoPixel Jewel',
            'diameter': {
                'outer': 0.9,
                'px': 0.63
            },
            'type': 'jewel'
        },
        '12': {
            'name': 'NeoPixel 12-LED Ring',
            'diameter': {
                'inner': 0.92,
                'outer': 1.45,
                'px': 1.16
            },
            'type': 'ring'
        },
        '16': {
            'name': 'NeoPixel 16-LED Ring',
            'diameter': {
                'inner': 1.25,
                'outer': 1.75,
                'px': 1.49
            },
            'type': 'ring'
        },
        '24': {
            'name': 'NeoPixel 24-LED Ring',
            'diameter': {
                'inner': 2.06,
                'outer': 2.58,
                'px': 2.3
            },
            'type': 'ring'
        }
    }
    INCH_CM = 2.54
    pixels_xy = dict()
    for key in rings:
        try:
            print('<RingValid-' + str(key) + '>')
            ring = NEOPIXEL_LED_RINGS[str(key)]
            angle = 0
            ring_xy_list = list()
            outer_pixels = int(key)
            if ring['type'] == 'jewel':
                ring_xy_list.append((0, 0))  # Add the central Pixel.
                outer_pixels -= 1
            dist_px_px = 360 / float(outer_pixels)  # In degree.
            phase = float(rings[key])
            radius = ring['diameter']['px'] / 2.0
            for i in range(outer_pixels):
                pixel_x = cos(radians(angle + phase)) * radius
                pixel_y = sin(radians(angle + phase)) * radius
                if cm:
                    pixel_x = INCH_CM * pixel_x
                    pixel_y = INCH_CM * pixel_y
                xy = tuple([round(pixel_x, 2), round(pixel_y, 2)])
                angle += dist_px_px
                ring_xy_list.append(xy)
            pixels_xy[str(key)] = ring_xy_list
        except KeyError:
            print('<RingInvalid-' + str(key) + '>')
    return pixels_xy

In [ ]:
def get_wave_vectors(xy_list, height):
    """Calculates the incident wave vectors based on the LED positions
    and the distance between the sample and the lamp. Holds the unit of
    length.
    """
    # Operate on xy_list (list of tuples) as numpy.ndarray
    xy_list_array = np.array(xy_list)
    xy_list_array = xy_list_array / float(height)
    xy_list_array = np.arctan(xy_list_array)
    xy_list_array = np.sin(xy_list_array)
    xy_list_array *= -1
    # Convert 2-dimensional array into a list of tuples.
    k_xy_tuple_list = list(map(lambda xy: tuple(xy), list(xy_list_array)))
    return k_xy_tuple_list